# Encoding

In [1]:
# Encode etmek değişkenlerin temsil sekilleri ile alakalı değişiklik yapmak demektir.

        Encode etmek aşağıdaki gibi değişkenlere onları ifade edecek yeni değişkenler atamaktır.

<p align="center" >    
    <img src = "grafikler/label_encoding1.png" width= "300" />
</p>

        Aşağıdaki gibi eğitim durumu değişkeni tanımlanmış olsun, burada açıktır ki "Pre-School" en düşük eğitim seviyesi ike PhD en yüksek eğitim seviyesidir. Yani veri sadece kategorik değil aynı zamanda ordinaldir. Yani kendi içinde sıralıdır.

        Yani verimizi bir sağ sütundaki gibi encode etmek doğru bir yaklaşımdır. Zira "0" olarak ifade edilen kategori zaten en düşük seviye eğitimi ifade etmekte iken, 5 olarak ifade edilen kategori en yüksek eğitimi ifade etmektedir.

<p align="center" >    
    <img src = "grafikler/label_encoding2.png" width= "300" />
</p>

        Aşağıdaki gibi takımlar değişkeni tanımlanmış olsun. Burada bir takımın diğer takım üzerinde bir üstünlüğü bulunmamaktadır. Yani veri hem kategorik hem de nominal bir veri türüdür.

        Dolayısıyla verimizi yukarıdaki gibi sınıflamak doğru değildir. Bunun için "One Hot Encoding" kullanacağız. Bu terimi de notebook'un ilerleyen bölümlerinde açıklayacağız.

<p align="center" >    
    <img src = "grafikler/label_encoding3.png" width= "300" />
</p>

In [2]:
# Kütüphanelerimizi import edelim;

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# !pip install missingno
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

In [3]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
# Önceden yazmış olduğumuz fonksiyonları tanımlayalım;

def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car


## Label Encoding

In [5]:
def load_application_train():
    data = pd.read_csv("datasets/application_train.csv")
    return data

def load():
    data = pd.read_csv("datasets/titanic.csv")
    return data

In [6]:
# Titanic veri setimizi çağıralım;

df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [7]:
# Binary Encoding ise label encoding'in iki sınıflı halidir.

# Şimdi bir kategorik değişken seçelim ve label (binary) endoce edelim.

df["Sex"].head()

0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object

In [8]:
# "sex" değişkenini binary encode etmek için;

le = LabelEncoder() # LabelEncoder() nesnemizi getiriyoruz.

# Bu nesneyi fit_transform metodu ile "sex" değişkenine uygulamamız gerekmektedir.

le.fit_transform(df["Sex"])[0:5]

# Burada alfabetik sıraya göre bir değer atama işlemi yapılmaktadır. 
# Female'deki f harfi m'den önce geldiği için "Female" : 0, "Male" : 1 olara kodlanmıştır.

array([1, 0, 0, 0, 1])

In [9]:
# Sadece binary encoding olarak değil daha çok sınıfı olan bir değişkende dönüşüm yaptığımızda 
# ve sınıfları bilme ihtiyacı hissettiğimizde ise;

le.inverse_transform([0, 1])

# kodu ile sınıfların karşılık geldiği sayısal değerleri öğrenebiliriz.

array(['female', 'male'], dtype=object)

In [10]:
# Bir binary encoder fonksiyonu tanımlamak istersek;

def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe

In [11]:
# Şimdi titenic verisini yeniden import edelim;

df = load()

In [12]:
# Yukarıdaki formülü uygulayacağız lakin elimizde yüzlerce değişken var ise bunu nasıl ölçekleyebiliriz;
# Bunu ölçekleyebilmek için binary sütunları seçebilmemiz gerekir,
# Bunun için de bir kod yazmak istersek;

binary_cols = [col for col in df.columns if df[col].dtype not in [int, float]
               and df[col].nunique() == 2]

binary_cols

['Sex']

        Bir farkındalık !

        ".nunique()" metodu NaN ifadeleri bir sınıf olarak görmezken ".unique()" metodu NaN ifafdeleri bir sınıf olarak görür.

In [13]:
print(df["Embarked"].nunique())
print(df["Embarked"].unique())
print(len(df["Embarked"].unique()))

3
['S' 'C' 'Q' nan]
4


In [14]:
# Yukarıda bir tane binary_cols çıktı lakin birden fazla veya onlarca binary_cols çıkmış olsaydı bir döngü kullanmam gerekirdi;

for col in binary_cols:
    label_encoder(df, col)

In [15]:
# Veri setimize bir bakalım;

df.head()

# Cinsiyet değişkeni artık encode edilmiş oldu

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.000,0,0,373450,8.050,NaN,S


In [16]:
# Daha büyük bir veri setinde bu işlemleri yapmak istersek;

df = load_application_train()
df.shape

# 122 değişken bulunmaktadır.

(307511, 122)

In [17]:
# .head() atarak da binary değişkeni bulmak çok zor.

df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.000,406597.500,24700.500,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,1.000
1,100003,0,Cash loans,F,N,N,0,270000.000,1293502.500,35698.500,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000
2,100004,0,Revolving loans,M,Y,Y,0,67500.000,135000.000,6750.000,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000
3,100006,0,Cash loans,F,N,Y,0,135000.000,312682.500,29686.500,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.000,513000.000,21865.500,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000


In [18]:
# binary_cols'u yeniden çağırdık;

binary_cols = [col for col in df.columns if df[col].dtype not in [int, float]
               and df[col].nunique() == 2]

binary_cols

['NAME_CONTRACT_TYPE',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'EMERGENCYSTATE_MODE']

In [19]:
# veri setini binary olan değişkenlerle inceleyelim;

df[binary_cols].head()

,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,EMERGENCYSTATE_MODE
0,Cash loans,N,Y,No
1,Cash loans,N,N,No
2,Revolving loans,Y,Y,NaN
3,Cash loans,N,Y,NaN
4,Cash loans,N,Y,NaN


In [20]:
# Veri setini döngü içerisine sokalım;

for col in binary_cols:
    label_encoder(df, col)

In [21]:
# veri setimize bir kez daha bakalım;

df[binary_cols].head()

,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,EMERGENCYSTATE_MODE
0,0,0,1,0
1,0,0,0,0
2,1,1,1,2
3,0,0,1,2
4,0,0,1,2


        ÖNEMLİ BİR NOT!!!

        Yukarıda binary olan değişkenleri 0 ve 1 formatına getirdik. Lakin 3. sütunda "2" değeri görüyoruz. Bu eksik değerlere 2 atandığını gösterir. Bunu bilmemiz gerekir ve unutmamalıyız.


## One Hot Encoding

In [22]:
# Label encoding'e benzese de kategorik sınıflar arasında bir fark yok ise, 
# bunlar arasında bir sıralama oluşturmak bir fark oluşturmak bir ölçüm problemi oluşturabilir.

        Label encoding ölçüm problemi oluşturma potansiyeli taşıdığından, kategorik sınıflar arasında fark yoksa aşağıdaki gibi one hot encoding yapılır.

<p align="center" >    
    <img src = "grafikler/one_hot_encoding.png" width= "800" />
</p>

        Ayrıca one hot encoding yapılırken dummy variable tuzağına düşmemek için ilk değişken drop edilir.

In [23]:
df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [24]:
df["Embarked"].value_counts()

# embarked değişkeninin 3 sınıfı var ve bunlar one hot encoding için uygun.

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [25]:
# Embarked değişkenini one hot encode edelim;

pd.get_dummies(df, columns=["Embarked"]).head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,0,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,0,0,1


In [26]:
# Dummy tuzağına düşmemek için ise;

pd.get_dummies(df, columns=["Embarked"], drop_first=True).head()

# İlk sınıfı (kurtulacağı sınıfı) ise alfebetik sıraya göre seçmektedir.

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,0,1


In [27]:
# Eksik değerler için de bir sınıf oluşturulsun istersek;

pd.get_dummies(df, columns=["Embarked"], dummy_na=True).head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,0,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,1,0,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,0,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,0,0,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,0,0,1,0


In [28]:
# Aynı anda birden fazla değişkeni one hot encode edebiliriz;

pd.get_dummies(df, columns=["Sex", "Embarked"], drop_first=True).head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_male,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",22.000,1,0,A/5 21171,7.250,NaN,1,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.000,1,0,PC 17599,71.283,C85,0,0,0
2,3,1,3,"Heikkinen, Miss. Laina",26.000,0,0,STON/O2. 3101282,7.925,NaN,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.000,1,0,113803,53.100,C123,0,0,1
4,5,0,3,"Allen, Mr. William Henry",35.000,0,0,373450,8.050,NaN,1,0,1


In [29]:
# Bu işlemi bir fonksiyon haline getirmek istersek;

def one_hot_encoder(dataframe, categorical_cols, drop_first=True):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe 

In [30]:
# df ve listelerimize bir reset atalım;

df = load()
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 891
Variables: 12
cat_cols: 6
num_cols: 3
cat_but_car: 3
num_but_cat: 4


In [31]:
# One Hot encoding yapacağımız sütunları bulalım;

ohe_cols = [col for col in df.columns if 10 >= df[col].nunique() > 2]
ohe_cols

['Pclass', 'SibSp', 'Parch', 'Embarked']

In [32]:
# Fonksiyonumuzu bir üstteki satırda bulduğumuz sütunlarla birlikte kullanalım;

one_hot_encoder(df, ohe_cols).head()

,PassengerId,Survived,Name,Sex,Age,Ticket,Fare,Cabin,Pclass_2,Pclass_3,...,SibSp_5,SibSp_8,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_Q,Embarked_S
0,1,0,"Braund, Mr. Owen Harris",male,22.000,A/5 21171,7.250,NaN,0,1,...,0,0,0,0,0,0,0,0,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,PC 17599,71.283,C85,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,"Heikkinen, Miss. Laina",female,26.000,STON/O2. 3101282,7.925,NaN,0,1,...,0,0,0,0,0,0,0,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,113803,53.100,C123,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,"Allen, Mr. William Henry",male,35.000,373450,8.050,NaN,0,1,...,0,0,0,0,0,0,0,0,0,1


## Rare Encoding

        Ölçüm değeri taşıdığına inanmadığımız belirli bir eşik değerden daha az frekans gösteren verileri bir araya toplayarak yeni sınıf yapısı oluşturulmasına rare encoding denir.

        Aşağıdaki tabloya bakıldığındaa L şehri için bir sütun oluşturmanın mantığı tartışılabilir. Frekansı bir olan tek şehir için yüzlerce başka değişken değeri 0 olan bir sütun oluşurulmasını sorguluyoruz.

<p align="center" >    
    <img src = "grafikler/rare_encoding.png" width= "800" />
</p>

In [33]:
# Rare encoding 3 aşama ile yapılacaktır;

# 1. Kategorik değişkenlerin azlık çokluk durumunun analiz edilmesi.
# 2. Rare kategoriler ile bağımlı değişken arasındaki ilişkinin analiz edilmesi.
# 3. Rare encoder fonksiyonunun yazılması.

In [34]:
###################
# 1. Kategorik değişkenlerin azlık çokluk durumunun analiz edilmesi.
###################

In [35]:
df = load_application_train()
df["NAME_EDUCATION_TYPE"].value_counts()

# Aşağıda görüldüğü üzere son iki değişkenin frekanslarını yeterince fazla olmadığı şeklinde yorumlayabiliriz.

Secondary / secondary special    218391
Higher education                  74863
Incomplete higher                 10277
Lower secondary                    3816
Academic degree                     164
Name: NAME_EDUCATION_TYPE, dtype: int64

In [36]:
# Kategorik değişkenlerimizi bir seçelim;

cat_cols, num_cols, cat_but_car = grab_col_names(df)
cat_cols

Observations: 307511
Variables: 122
cat_cols: 54
num_cols: 67
cat_but_car: 1
num_but_cat: 39


['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE',
 'TARGET',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOC

In [37]:
# Şimdi daha öncelerden de tanıdığımız bir fonksiyon tanımlayalım;
# Bu fonksiyon bize bir df içerisindeki kategorik değişkenlerin sınıflarını ve bu sınıfların oranlarını getirsin;

def cat_summary(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show()

In [38]:
for col in cat_cols:
    cat_summary(df, col)

                 NAME_CONTRACT_TYPE  Ratio
Cash loans                   278232 90.479
Revolving loans               29279  9.521
##########################################
     CODE_GENDER  Ratio
F         202448 65.834
M         105059 34.164
XNA            4  0.001
##########################################
   FLAG_OWN_CAR  Ratio
N        202924 65.989
Y        104587 34.011
##########################################
   FLAG_OWN_REALTY  Ratio
Y           213312 69.367
N            94199 30.633
##########################################
                 NAME_TYPE_SUITE  Ratio
Unaccompanied             248526 80.819
Family                     40149 13.056
Spouse, partner            11370  3.697
Children                    3267  1.062
Other_B                     1770  0.576
Other_A                      866  0.282
Group of people              271  0.088
##########################################
                      NAME_INCOME_TYPE  Ratio
Working                         158774 51.632
C

In [39]:
cat_summary(df, "NAME_FAMILY_STATUS")

# Aşağıdaki verinin içerisinde bulunma oranı yüzbinde 1 olan "Unknown" sınıfı için bir label oluşturmak doğru bir yaklaşım olmayabilir.
# Bu "NAME_FAMILY_STATUS" değişkeni gibi bir çok değişken olduğunu görebiliyoruz.

                      NAME_FAMILY_STATUS  Ratio
Married                           196432 63.878
Single / not married               45444 14.778
Civil marriage                     29775  9.683
Separated                          19770  6.429
Widow                              16088  5.232
Unknown                                2  0.001
##########################################


In [40]:
len(cat_cols)

54

        Yukarıda görüldüğü üzere 54 adet kategorik değişken var, biz bunların tamamını one hot encode edersek işimiz çok fazla zorlaşabilir. Amacımız karmaşık problemler çıkarmak değil, en basit modeli uygulamak.

In [41]:
###################
# 2. Rare kategoriler ile bağımlı değişken arasındaki ilişkinin analiz edilmesi.
###################

In [42]:
df.head()

# Verinin bağımlı değişkeni target'dır ve 1 olması default'ı 0 olması ise kredilerin zamanında ödendiğini ifade eder.

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.000,406597.500,24700.500,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,1.000
1,100003,0,Cash loans,F,N,N,0,270000.000,1293502.500,35698.500,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000
2,100004,0,Revolving loans,M,Y,Y,0,67500.000,135000.000,6750.000,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000
3,100006,0,Cash loans,F,N,Y,0,135000.000,312682.500,29686.500,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.000,513000.000,21865.500,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000


In [43]:
# Veri setimize ait "NAME_INCOME_TYPE" sütununu inceleyelim;

df["NAME_INCOME_TYPE"].value_counts()

Working                 158774
Commercial associate     71617
Pensioner                55362
State servant            21703
Unemployed                  22
Student                     18
Businessman                 10
Maternity leave              5
Name: NAME_INCOME_TYPE, dtype: int64

In [44]:
# "NAME_INCOME_TYPE" değişkeninin altındaki meslek gruplarının kredileri ödeme durumunu oransal olarak kontrol edelim;

df.groupby("NAME_INCOME_TYPE")["TARGET"].mean()

# 1'e yaklaştıkça kredinin ödenememesini ifade eder.

# Businessman'lerin kredi ödemelerinde sorun çekmediklerini gözlemlerken, 
# işsiz ve doğum iznindekilerin temerrüde daha çok düştükleri gözlemlenmektedir.

NAME_INCOME_TYPE
Businessman            0.000
Commercial associate   0.075
Maternity leave        0.400
Pensioner              0.054
State servant          0.058
Student                0.000
Unemployed             0.364
Working                0.096
Name: TARGET, dtype: float64

        Yukarıda "Businessman"lerin temerrüde düşmediklerini söylüyoruz lakin veri içerisindeki görülme frekansı 10, doğum iznindekiler hakkında bir çıkarımda bulunduk lakin veride yeterince temsil ediliyor mu sorusuna net bir cevap vermek pek de mümkün değil.

        Lakin bunlar arasında bir fark olduğu da açık, maternity leave olanlar ile business man olanlar arasında temerrüde düşme açısından da bir fark bulunuyor.

        Bunları aynı rare variable altında toplamak mantıklı mı bu soruyu da kendimize soruyor olacağız. Bunları aynı değişken altında toplamak da anlamlandırılma açısından bir noktaya varmayabilir ve veri setinde bir gürültü oluşuturuyor olabilir.

        Ama şimdilik biz aynı değişken (rare variable) altında topluyor olacağız. :)

In [45]:
# Verimizde nadir görülen sınıfları bulmak adına bir fonksiyon tanımlıyor olacağız;

def rare_analyser(dataframe, target, cat_cols):
    for col in cat_cols:
        print(col, ":", len(dataframe[col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO": dataframe[col].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(col)[target].mean()}), end="\n\n\n")

In [46]:
# Yukarıdaki kodun ilk kısmının açıklaması;

print(df["NAME_CONTRACT_TYPE"].value_counts())
print("###########################################")
print(len(df["NAME_CONTRACT_TYPE"].value_counts()))

Cash loans         278232
Revolving loans     29279
Name: NAME_CONTRACT_TYPE, dtype: int64
###########################################
2


In [47]:
rare_analyser(df, "TARGET", cat_cols)

NAME_CONTRACT_TYPE : 2
                  COUNT  RATIO  TARGET_MEAN
Cash loans       278232  0.905        0.083
Revolving loans   29279  0.095        0.055


CODE_GENDER : 3
      COUNT  RATIO  TARGET_MEAN
F    202448  0.658        0.070
M    105059  0.342        0.101
XNA       4  0.000        0.000


FLAG_OWN_CAR : 2
    COUNT  RATIO  TARGET_MEAN
N  202924  0.660        0.085
Y  104587  0.340        0.072


FLAG_OWN_REALTY : 2
    COUNT  RATIO  TARGET_MEAN
N   94199  0.306        0.083
Y  213312  0.694        0.080


NAME_TYPE_SUITE : 7
                  COUNT  RATIO  TARGET_MEAN
Children           3267  0.011        0.074
Family            40149  0.131        0.075
Group of people     271  0.001        0.085
Other_A             866  0.003        0.088
Other_B            1770  0.006        0.098
Spouse, partner   11370  0.037        0.079
Unaccompanied    248526  0.808        0.082


NAME_INCOME_TYPE : 8
                       COUNT  RATIO  TARGET_MEAN
Businessman               10  0.

In [48]:
rare_analyser(df, "TARGET", ["DEF_60_CNT_SOCIAL_CIRCLE"])

# Aşağıda görüldüğü üzere veri içinde temsil özelliği bulunmayabilecek bir çok sınıf vardır.

DEF_60_CNT_SOCIAL_CIRCLE : 9
         COUNT  RATIO  TARGET_MEAN
0.000   280721  0.913        0.078
1.000    21841  0.071        0.105
2.000     3170  0.010        0.121
3.000      598  0.002        0.159
4.000      135  0.000        0.111
5.000       20  0.000        0.150
6.000        3  0.000        0.000
7.000        1  0.000        0.000
24.000       1  0.000        0.000




        Uygulama kapsamında %1'in altında görülme sıklığını bulunan sınıfları bir araya getirmeyi tercih ediyor olacağız.

In [49]:
#############################################
# 3. Rare encoder'ın yazılması.
#############################################

In [50]:
# Rare encoder fonksiyonunu tanımlıyoruz;

def rare_encoder(dataframe, rare_perc):

    # Üzerinde değişiklik yapılacağından dolayı dataframe'mimizin bir kopyasını alalım.
    temp_df = dataframe.copy()

    # rare_columns listesi tanımlayalım ve yakalayacağımız rare değişkenleri bu listede tutalım.
    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'    # değişkenin tipi object (kategorik) ise ve ...
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]
                    # ... veri içerisinde görülme sıklığı belirlenen "rare_perc" oranından küçük grup barındıran değişkenlerin seçilme işlemi
                    # Yukarıda ".any()" metodu da bulunmaktadır, içinde var ise listeye almasını söylüyoruz.


    for var in rare_columns:    # içerisinde görülme sıklığı %1'in altıda grup barındıran rare column'larda gez,
        tmp = temp_df[var].value_counts() / len(temp_df) # Bu değişken/sütunlar içerisindeki grupların yüzdelerini hesapla,
        rare_labels = tmp[tmp < rare_perc].index    # Bu hesaplanan yüzdeler "rare_perc"ten küçük ise bunların indeksini al,
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var]) # indeksi alınan yerdeki değişkene rare yaz diyoruz.

    return temp_df

In [51]:
# temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var]) kodu ise excel'deki IF gibi çalışır.

# np.where = if, temp_df[var].isin(rare_labels) = koşul burada diyoruz ki ilgili değişken rare_label içinde ise, 
# 'Rare' yaz, değil ise "emp_df[var]" diyerek olduğu gibi bırakmasını söylüyoruz.

## Yukarıdaki "rare_encoder" kodunun açıklaması

In [52]:
df_deneme = load_application_train()
df_deneme

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.000,406597.500,24700.500,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,1.000
1,100003,0,Cash loans,F,N,N,0,270000.000,1293502.500,35698.500,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000
2,100004,0,Revolving loans,M,Y,Y,0,67500.000,135000.000,6750.000,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000
3,100006,0,Cash loans,F,N,Y,0,135000.000,312682.500,29686.500,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.000,513000.000,21865.500,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.000,254700.000,27558.000,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.000,269550.000,12001.500,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.000,677664.000,29979.000,...,0,0,0,0,1.000,0.000,0.000,1.000,0.000,1.000
307509,456254,1,Cash loans,F,N,Y,0,171000.000,370107.000,20205.000,...,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000


In [53]:
rare_analyser(df_deneme, "TARGET", ["DEF_60_CNT_SOCIAL_CIRCLE"])

DEF_60_CNT_SOCIAL_CIRCLE : 9
         COUNT  RATIO  TARGET_MEAN
0.000   280721  0.913        0.078
1.000    21841  0.071        0.105
2.000     3170  0.010        0.121
3.000      598  0.002        0.159
4.000      135  0.000        0.111
5.000       20  0.000        0.150
6.000        3  0.000        0.000
7.000        1  0.000        0.000
24.000       1  0.000        0.000




In [54]:
rare_columns_2 = [col for col in df_deneme.columns if df_deneme[col].dtypes == 'O'
                    and (df_deneme[col].value_counts() / len(df_deneme) < 0.01).any(axis=None)]
rare_columns_2

['CODE_GENDER',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'ORGANIZATION_TYPE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [55]:
for var in rare_columns_2:
    tmp = df_deneme[var].value_counts() / len(df_deneme)

In [56]:
tmp

No    0.518
Yes   0.008
Name: EMERGENCYSTATE_MODE, dtype: float64

In [57]:
rare_labels = tmp[tmp < 0.01].index
rare_labels

Index(['Yes'], dtype='object')

In [58]:
df_deneme["EMERGENCYSTATE_MODE"] = np.where(df_deneme["EMERGENCYSTATE_MODE"].isin(rare_labels), 'Rare', df_deneme["EMERGENCYSTATE_MODE"])

In [59]:
df_deneme["EMERGENCYSTATE_MODE"].value_counts()/ len(df_deneme)

No     0.518
Rare   0.008
Name: EMERGENCYSTATE_MODE, dtype: float64

In [60]:
for var in rare_columns_2:    
        tmp = df_deneme[var].value_counts() / len(df_deneme) 
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(df_deneme[var].isin(rare_labels), 'Rare', df_deneme[var])

NameError: name 'rare_perc' is not defined

## *** Açıklama bitişi ***

In [ ]:
new_df = rare_encoder(df, 0.01)
new_df

## Özellik Ölçeklendirme (Feature Scaling)

In [ ]:
# Özellik ölçeklendirmedeki amaçlardan birisi değişkenler arasındaki ölçüm farklılıklarını gidermektir.
# Yani kullanılacak olan modellerin değişkenlere eşit şartlar arasında yaklaşmasını sağlamaya çalışmaktır.

<p align="center" >    
    <img src = "grafikler/feauture_scaling.png"/>
</p>

In [ ]:
# Standartlaştırma metodları

In [ ]:
###################
# StandardScaler: Klasik standartlaştırma. 
# Ortalamayı çıkar, standart sapmaya böl. z = (x - u) / s
###################

In [ ]:
df = load()
ss = StandardScaler()
df["Age_standard_scaler"] = ss.fit_transform(df[["Age"]])
df.head()

In [ ]:
###################
# RobustScaler: Medyanı çıkar iqr'a böl.
# Yukarıda standartlaştırırken, ortalama da standart sapma da aykırı değerlerden etkilenen metrikler olduğu unutulmamalıdır.
# Robust'da medyandan iqr çıkartarak hem merkezi eğilim ölçülerini göz önünde bulunduruyoruz, hem de standarlaştırma işlemi yapıyoruz.
###################

In [ ]:
rs = RobustScaler()
df["Age_robuts_scaler"] = rs.fit_transform(df[["Age"]])
df.describe().T

In [ ]:
###################
# MinMaxScaler: Verilen 2 değer arasında değişken dönüşümü
###################

# X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
# X_scaled = X_std * (max - min) + min

In [ ]:
mms = MinMaxScaler()
df["Age_min_max_scaler"] = mms.fit_transform(df[["Age"]])

In [ ]:
df.describe().T

# Aşağıdaki tabloda "Age_standard_scaler" değişkeni incelendiğinde ortalamanın 0 standart sapmanın ise 1 olduğunu görebiliyoruz.
# Yine "Age_robuts_scaler" değişkenine bakılırsa ortalama ve std. dev.'in değiştiğini görüyoruz.

# Lakin "Age_standard_scaler" ve "Age_robuts_scaler" kıyaslandığında "Age_robuts_scaler"ın min ve max değerleri arasındaki farkın,
# daha az olduğu görülmektedir. Bu da aykırı değerlerden daha az etkilenmesinden kaynaklanmaktadır.

# "Age_min_max_scaler" değişkeninde ise min. değer 0 iken max. değer 1'dir.

In [ ]:
df.head()

In [ ]:
# İçinde age geçen değişkenleri bir yakalayalım;

age_cols = [col for col in df.columns if "Age" in col]
age_cols

In [ ]:
# Aşağıda ilgili değişkenler için grafik çizdirecek bir fonksiyon tanımlayalım;

def num_summary(dataframe, numerical_col, plot=False):
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[numerical_col].describe(quantiles).T)

    if plot:
        dataframe[numerical_col].hist(bins=20)
        plt.xlabel(numerical_col)
        plt.title(numerical_col)
        plt.show()

In [ ]:
for col in age_cols:
    num_summary(df, col, plot=True)

        Yukarıda görülmektedir ki değişkenler üzerinde uyguladığımız değişim dağılımı değiştirmemektedir.
        Yaptığımız işlem değişkenin yapısını bozmak değildir, yapısını koruyacak şekilde ifade ediliş tarzlarını değiştirme yani standartlaştırma işlemidir.

        Yani değişkenin taşıdığı bilgi bozulmamıştır.

In [ ]:
###################
# Numeric to Categorical: Sayısal Değişkenleri Kateorik Değişkenlere Çevirme
# Binning
###################

In [ ]:
df["Age_qcut"] = pd.qcut(df['Age'], 5)
df.head()

# böleceğimiz değişkenin etiketlerini biliyorsak, metod içerisine "label=" argümanı göndeririz.